In [ ]:
import numpy as np
import scipy.signal
from scipy.linalg import block_diag
from scipy.optimize import minimize
from qpsolvers import solve_qp
import qpsolvers
import pandas as pd
import matplotlib.pyplot as plt
import cProfile
import time

In [ ]:
# Prediction Horizon
N = 2

In [ ]:
# Circuit Parameters
R1 = 8.9
C1 = 220e-6
L1 = 1e-4
v_i = 12
Ts = 2e-5

# Continuous-Time State Space Equations
Ac = np.array([[0, -1/L1], [1/C1, -1/(R1*C1)]])
Bc = np.array([[1/L1], [0]])
Cc = np.eye(Ac.shape[0])

# Discrete-Time State Space Equations
Ad, Bd, _, _, _ = scipy.signal.cont2discrete((Ac, Bc, Cc, 0), Ts)
inv_Ad = np.linalg.inv(Ad)

# State space solver x' = sys = Ax+Bu, y = Cx+D
nx, _ = Ad.shape
_, nu = Bd.shape
ny, _ = Cc.shape

Ae = np.block([[Ad, np.zeros((nx, ny))],
               [Cc @ Ad, np.eye(ny)]])
Be = np.block([[Bd],
              [Cc @ Bd]])
Ce = np.block([[np.zeros((ny, nx)), np.eye(ny)]])

nxn, _ = Ae.shape
_, ndu = Be.shape

# N = 5 # Prediction Horizon

Phi = np.zeros((N * ny, nxn))

# Create Gamma matrix
Gamma = np.zeros((N * ny, N * ndu))

# Populate both matrices
for i in range(1, N + 1):
    # Phi
    row_init_phi = 1 + (i - 1) * ny
    row_end_phi = row_init_phi + (ny - 1)
    Phi[row_init_phi-1:row_end_phi, :] = Ce @ np.linalg.matrix_power(Ae, i)

    # Gamma
    row_init_gamma = 1 + (i - 1) * ny
    row_end_gamma = row_init_gamma + (ny - 1)
    for j in range(1, i + 1):
        col_init = 1 + (j - 1) * ndu
        col_end = col_init + (ndu - 1)
        Gamma[row_init_gamma-1:row_end_gamma, col_init-1:col_end] = Ce @ np.linalg.matrix_power(Ae, i - j) @ Be


# Performance matrices
R = 0.02 * np.eye(ndu)  # Δu (input difference)
outputsForCost = [0, 1]  # Select which outputs should be tracked in cost function
Q = 100 * np.diag(outputsForCost)  # y-r (reference tracking error)
P = 1 * Q  # y_N-r_N (steady-state reference tracking error)

# Weight matrices
Omega = block_diag(*[Q]*(N-1), P)  # y-r from 1 to N
Psi = block_diag(*[R]*N)  # Δu from 0 to N-1

# Cost Function
# J(x_n(k), ΔU_k) = 1/2*ΔU_k*G*ΔU_k + ΔU*F*(Phi*x_n(k) - R_k)
# where
# R_k = References vector computed online
G = 2 * (Psi + Gamma.T @ Omega @ Gamma)
F = 2 * Gamma.T @ Omega

# QP model
# min 1/2 * x_QP' * Q * x_QP + q*x_QP
Q = (G + G.T) / 2  # Positive semidefinite

# q = F*(Phi*x_old - R_k)
# quadprog(Q,q,0,0,.....)

# Constraints
# Output and state (as part of output) constraints NEED TO UPDATE
ymin = np.array([[0], [0]])  # Column vector for i_L and v_C
ymax = np.array([[10], [50]])  # i_L, v_C

# Control input (should be vertical vector)[uD; uQ] <- how does this work?
umin = np.array([0])
umax = np.array([20])

# Control input rate (should be vertical vector)[ΔuD; ΔuQ]
dumin = np.array([-1000])
dumax = np.array([1000])

def MPC(ref, Cc, Q, F, Phi, Gamma, N, ndu, umin, umax, ymin, ymax, dumin, dumax, state, state_old, Vin_old):
    x_n_old = np.vstack((state-state_old, Cc@state))
    nextStepRef = np.vstack((0, ref))
    R_k = np.kron(np.ones((N,1)), nextStepRef)
    I_constraints = np.eye(ndu)
    C2_constraints = np.kron(np.tril(np.ones((N))), I_constraints)
    M1_constraints = np.vstack((-C2_constraints, C2_constraints))
    C1_constraints = np.kron(np.ones((N,1)), I_constraints)
    Umin_constraints = np.kron(np.ones((N,1)), umin)
    Umax_constraints = np.kron(np.ones((N,1)), umax)
    N1_constraints = np.vstack((-Umin_constraints + np.dot(C1_constraints, Vin_old), Umax_constraints - np.dot(C1_constraints, Vin_old)))
    I_horizon_constraints = np.eye(N*ndu)
    M2_constraints = np.vstack((-I_horizon_constraints, I_horizon_constraints))
    DeltaUmin_constraints = np.kron(np.ones((N,1)), dumin)
    DeltaUmax_constraints = np.kron(np.ones((N,1)), dumax)
    N2_constraints = np.vstack((-DeltaUmin_constraints, DeltaUmax_constraints))
    M3_constraints = np.vstack((-Gamma, Gamma))
    Ymin_constraints = np.kron(np.ones((N,1)), ymin)
    Ymax_constraints = np.kron(np.ones((N,1)), ymax)
    N3_constraints = np.vstack((-Ymin_constraints + Phi @ x_n_old, Ymax_constraints - Phi @ x_n_old))
    M_constraint = np.vstack((M1_constraints, M2_constraints, M3_constraints))
    N_constraint = np.vstack((N1_constraints, N2_constraints, N3_constraints))
    q = np.dot(F, np.dot(Phi, x_n_old) - R_k)
    test = np.dot(F, np.dot(Phi, x_n_old))
    options = {'disp': False}

    du = qpsolvers.solve_qp(Q, q, M_constraint, N_constraint, solver="cvxopt")
    
    if du is None:
      print("Solver failed to find a feasible solution.")
    else:
      Vin = du[0] + Vin_old
      Vin_old = Vin
      return Vin

In [ ]:
state = np.array([[0],[0]])
Vin_old = 0
ref = 12
state_old = inv_Ad @ (state - np.dot(Bd, Vin_old))

state_history = []
prediction_history = []

prediction_history.append([Vin])
state_history.append(state.tolist())

for i in range(100):

    Vin = MPC(ref, Cc, Q, F, Phi, Gamma, N, ndu, umin, umax, ymin, ymax, dumin, dumax, state, state_old, Vin_old)
   
    try:
        if Vin is None:
            raise ValueError("Solver failed to find a feasible solution")
        
    except Exception as e:
        print("Exception occurred:", e)
        Vin = state_old[1].item()
        
    state_old = state    
    state = np.dot(Ad,state) + np.dot(Bd,Vin)
    
    prediction_history.append([Vin])
    state_history.append(state.tolist())
    
    Vin_old = Vin

state_history = np.array(state_history)
prediction_history = np.array(prediction_history)

In [ ]:
state = np.array([[0], [0]])
Vin_old = 0
ref = 12
inv_Ad = np.linalg.inv(Ad)
state_old = inv_Ad @ (state - np.dot(Bd, Vin_old))

state_history_10 = []
prediction_history_10 = []

prediction_history_10.append([Vin_old])
state_history_10.append(state.tolist())
mpc_times = []

for i in range(100):
    start_time = time.time()
    
    Vin = MPC(ref, Cc, Q, F, Phi, Gamma, N, ndu, umin, umax, ymin, ymax, dumin, dumax, state, state_old, Vin_old)
    
    end_time = time.time() 
    mpc_times.append(end_time - start_time)
    
    try:
        if Vin is None:
            raise ValueError("Solver failed to find a feasible solution")
    except Exception as e:
        print("Exception occurred:", e)
        Vin = state_old[1].item()
        
    state_old = state    
    state = np.dot(Ad, state) + np.dot(Bd, Vin)
    
    prediction_history_10.append([Vin])
    state_history_10.append(state.tolist())
    
    Vin_old = Vin

state_history_10 = np.array(state_history_10)
prediction_history_10 = np.array(prediction_history_10)

total_time = sum(mpc_times)
average_time = total_time / len(mpc_times)

print(f"Prediction Horizon: {N}")
print(f"Total time taken for 100 MPC calls: {total_time:.6f} seconds")
print(f"Average time per MPC call: {average_time:.6f} seconds")

In [ ]:
time_steps = np.arange(len(state_history))

plt.figure(figsize=(10, 6))

plt.plot(time_steps, state_history_2[:, 0], label='State 1: i_L [A]')
plt.plot(time_steps, state_history_2[:, 1], label='State 1: v_C [V]')
plt.step(time_steps, prediction_history_2, label='Vin [V]', linestyle='--')

plt.xlabel('Iterations')
plt.ylabel('Value')
plt.legend()
plt.title('State Evolution and MPC Control Actions')

plt.tight_layout()
plt.grid(True)
plt.show()

final_state_flat = state_history[-1].flatten()
final_prediction_flat = prediction_history[-1].flatten()

final_state_data = {
    'Final State [A]; [V]': final_state_flat,
    'Final Prediction [V]': final_prediction_flat
}

final_state_df = pd.DataFrame(final_state_data, index=[f'Value {i+1}' for i in range(len(final_state_flat))])
display(final_state_df)

In [ ]:
def compute_settling_time(state_history, time_steps, tolerance_percentage, Ts):
    final_value_current = state_history[-1, 0]
    final_value_voltage = state_history[-1, 1]
    
    tolerance = tolerance_percentage * abs(final_value_voltage)
    settled = np.abs(state_history[:, 1] - final_value_voltage) <= tolerance
     
    for i in range(len(settled)):
        if all(settled[i:]):
            settling_time_index = i
            settling_time_seconds = time_steps[settling_time_index] * Ts
            break
    else:
        settling_time_index = None
        settling_time_seconds = None
        
    return settling_time_index, settling_time_seconds, final_value_voltage[0], final_value_current[0]

settling_time_index, settling_time_seconds, final_value_voltage, final_value_current = compute_settling_time(state_history, time_steps, 0.01, Ts)


data = {
    "Prediction Horizon": [N],
    "Settling Time [# Iterations]": [settling_time_index],
    "Settling Time [s]": [settling_time_seconds],
    "Final Voltage [V]": [final_value_voltage],
    "Final Current [A]": [final_value_current]
}

df = pd.DataFrame(data)
df_style = df.style.set_properties(**{'text-align': 'center'})
df_style = df_style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
display(df_style)